In [3]:
import sys
sys.path.append('../')

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from wrappers.bioc_wrapper import bioc_to_docs, bioc_to_relevances
from wrappers.pandas_wrapper import doc_to_pandas_tokens, annot_to_pandas, relevances_to_pandas, doc_to_pandas_docs
from pipelines.features_generator_docs import generate_features_dr
from pipelines.ml_config import MLConfig
from pipelines.ml import train_model, predict_with_model, evaluate_model, deserialize_mlconfig
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC

In [5]:
fp_train = '../datasets/PMtask_Triage_TrainingSet.xml'
docs_train = bioc_to_docs(fp_train)
relevances_train = bioc_to_relevances(fp_train, 'protein-protein')

In [6]:
df_train = doc_to_pandas_docs(docs_train)
df_train = relevances_to_pandas(df_train, relevances_train)

In [7]:
documents = df_train['Document']

In [8]:
df_train

,Document,Label
9685346,<data_structures.document.Document object at 0...,no
10364224,<data_structures.document.Document object at 0...,no
10688642,<data_structures.document.Document object at 0...,no
12059041,<data_structures.document.Document object at 0...,no
12897151,<data_structures.document.Document object at 0...,no
15294869,<data_structures.document.Document object at 0...,no
16612335,<data_structures.document.Document object at 0...,no
16909109,<data_structures.document.Document object at 0...,no
16939421,<data_structures.document.Document object at 0...,no
17050006,<data_structures.document.Document object at 0...,no


In [9]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
y_train = lb.fit_transform(df_train['Label'])
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [11]:
relevant_index_list = df_train[df_train['Label']=='yes'].index.tolist()
nonrelevant_index_list = df_train[df_train['Label']=='no'].index.tolist()

In [12]:
len(relevant_index_list)

1729

In [13]:
len(nonrelevant_index_list)

2353

In [54]:
for i in range(40):
    print(relevant_index_list[i])

18273060
22289178
19877579
17699613
16236155
11734542
23918659
20485264
19207211
22817896
20494986
8940099
20095964
11230138
21642453
7679113
17699755
18617892
20691684
9467941
20493856
10357812
19749176
12172031
12730685
18093972
23957209
9194564
10675337
11423549
16603186
19008858
16443932
18647389
11777938
21131964
22897854
21681199
20972459
17932487


In [55]:
for i in range(40):
    print(nonrelevant_index_list[i])

9685346
10364224
10688642
12059041
12897151
15294869
16612335
16909109
16939421
17050006
17069992
17088250
17101779
17102134
17105732
17135357
17179159
17182859
17200862
17203972
17215868
17220300
17227757
17229891
17243098
17296736
17317104
17332776
17353368
17371848
17372204
17386941
17389010
17438553
17442957
17462724
17496930
17509614
17510061
17516032


In [49]:
fp_test = '../datasets/PMtask_Triage_TestSet.xml'
docs_test = bioc_to_docs(fp_test)
relevances_test = bioc_to_relevances(fp_test, 'protein-protein')

In [50]:
df_test = doc_to_pandas_docs(docs_test)
df_test = relevances_to_pandas(df_test, relevances_test)

In [51]:
relevanttest_index_list = df_test[df_test['Label']=='yes'].index.tolist()
nonrelevanttest_index_list = df_test[df_test['Label']=='no'].index.tolist()

In [56]:
for i in range(50):
    print(relevanttest_index_list[i])
    print(nonrelevanttest_index_list[i])

9119005
10220326
8034624
9786929
17689488
9892652
12953057
16605263
9777370
25299333
10358088
12118074
10561590
12887828
12368318
10965042
10889020
10075741
15523651
8594202
24956593
9790601
11170396
11943812
21421369
7675451
14699067
26670244
14645919
16507992
11136978
17535917
18635949
17222866
26672965
16737965
8798490
23108402
11520787
10892746
20303977
16501109
24297439
11266465
8805302
16891660
9642250
11500501
12215524
24374182
12446613
20154136
10191253
17904127
9886379
15671037
16081517
11516166
9312046
12639967
16756945
24152121
20615419
12578370
12399472
26963794
15618221
18823366
20055437
10926931
8664285
19071118
15133049
17567696
22902837
9774384
10933782
9497377
10837138
25724843
7535778
28049826
15541764
15532026
8804434
25294941
9465038
9786899
23758859
12655090
17008717
17182855
8672451
12393906
22451851
12576545
27799292
15518587
9858607
10428802


In [ ]:
clf = SVC()
path_ml_model = 'default_svm_model.txt'
ml_config = MLConfig(clf, path_ml_model)

In [ ]:
df_features = generate_features_dr(df_train, ml_config)

In [17]:
from pipelines.features_generator_docs import test

In [18]:
test()

1


In [14]:
%reload_ext autoreload